We'll need some pretty standard stuff:

In [1]:
from petrify.space import Vector, Point, Plane
from petrify import tau, plane

Also, by default the renderer shows things head-on, so let's rotate them a little to get a better idea of what they look like.

In [2]:
def show(solid):
    rotated = (
        solid.rotate_around(tau / 8, Vector.basis.x)
             .rotate_around(tau / 8, Vector.basis.y)
    )
    return rotated.visualize()

Of course, the humble box is the literal building block of many pieces of solid geometry:

In [3]:
from petrify.solid import Box

show(Box(Point.origin, Vector(1, 1, 1)))

Preview(child=BufferGeometry(attributes={'normal': <BufferAttribute shape=(36, 3), dtype=float32>, 'position':…

And its round cousin, the cylinder:

In [4]:
from petrify.solid import Cylinder

show(Cylinder(Point.origin, Vector.basis.y * 3, 1.0, segments=20))

Preview(child=BufferGeometry(attributes={'normal': <BufferAttribute shape=(228, 3), dtype=float32>, 'position'…

Often, shapes defined in the plane need to be extruded directly to form useful solids:

In [5]:
from petrify.solid import Basis, PlanarPolygon, PolygonExtrusion

triangle = plane.Polygon([
    plane.Point(0, 0),
    plane.Point(0, 2),
    plane.Point(1, 1)
])

show(PolygonExtrusion(PlanarPolygon(Basis.unit, triangle), Vector(0, 0, 1)))

Preview(child=BufferGeometry(attributes={'normal': <BufferAttribute shape=(24, 3), dtype=float32>, 'position':…

You can also create solids by chaining arbitrary planar polygons:

In [6]:
from petrify.solid import Extrusion

parallelogram = plane.Polygon([
    plane.Point(0, 0),
    plane.Point(0, 1),
    plane.Point(1, 2),
    plane.Point(1, 1)
])
square = plane.Polygon([
    plane.Point(0, 0),
    plane.Point(0, 1),
    plane.Point(1, 1),
    plane.Point(1, 0)
])
dz = Vector.basis.z
show(Extrusion([
    PlanarPolygon(Basis.xy, parallelogram),
    PlanarPolygon(Basis.xy + dz, square),
    PlanarPolygon(Basis.xy + 2 * dz, parallelogram),
]))


Preview(child=BufferGeometry(attributes={'normal': <BufferAttribute shape=(60, 3), dtype=float32>, 'position':…

In [7]:
from petrify.solid import Collection

a = Box(Point.origin, Vector(1, 1, 1))
b = Box(Point.origin + Point(0.5, 0.5, 0.5), Vector(1, 1, 1))

Collection([a, b]).visualize(wireframe=True)

Preview(child=Mesh(geometry=BufferGeometry(attributes={'normal': <BufferAttribute shape=(72, 3), dtype=float32…

In [8]:
from petrify.solid import Union

Union([a, b]).visualize(wireframe=True)

Preview(child=Mesh(geometry=BufferGeometry(attributes={'normal': <BufferAttribute shape=(144, 3), dtype=float3…

In [9]:
(a - b).visualize(wireframe=True)

Preview(child=Mesh(geometry=BufferGeometry(attributes={'normal': <BufferAttribute shape=(90, 3), dtype=float32…

In [10]:
(a * b).visualize(wireframe=True)

Preview(child=Mesh(geometry=BufferGeometry(attributes={'normal': <BufferAttribute shape=(36, 3), dtype=float32…

In [11]:
import random
from petrify.solid import Union

def spin(shape):
    return (
        shape
            .rotate_around(random.uniform(0, tau / 4), Vector.basis.x)
            .rotate_around(random.uniform(0, tau / 4), Vector.basis.y)
            .rotate_around(random.uniform(0, tau / 4), Vector.basis.z)
    )

def delta(): return random.uniform(0.75, 1.25)

cubes = [
    spin(Box(
        Point.origin,
        random.uniform(1, 2) * Vector(1, 1, 1)
    )) + Point(x * delta(), y * delta(), delta())
    for x in range(8)
    for y in range(3)
]

show(Union(cubes))

Preview(child=BufferGeometry(attributes={'normal': <BufferAttribute shape=(15738, 3), dtype=float32>, 'positio…